In [ ]:
import zmq
import numpy as np
from fastplotlib import Plot

### Setup zmq subscriber client

In [ ]:
context = zmq.Context()
sub = context.socket(zmq.SUB)
sub.setsockopt(zmq.SUBSCRIBE, b"")

# keep only the most recent message
sub.setsockopt(zmq.CONFLATE, 1)

# address must match publisher in Processor actor
sub.connect("tcp://127.0.0.1:5555")

for testing things, benchmark zmq

In [ ]:
def get_buffer():
    """
    Gets the buffer from the publisher
    """
    try:
        b = sub.recv(zmq.NOBLOCK)
    except zmq.Again:
        pass
    else:
        return b
    
    return None

### Live plot that updates using the most recent message :D 

In [ ]:
plot = Plot()

# initialize image graphic with zeros
plot.add_image(
    np.zeros((512, 512), dtype=np.uint16),
    vmin=0,
    vmax=255,
    name="img"
)

def update_frame(p):
    # recieve memory with buffer
    buff = get_buffer()
    
    if buff is not None:
        # numpy array from buffer
        a = np.frombuffer(buff, dtype=np.uint32)
        ix = a[-1]
        # set graphic data
        p["img"].data = a[:-1].reshape(512, 512)
        p.set_title(f"frame: {ix}")

plot.add_animations(update_frame)

plot.show()

## **fastplotlib is non blocking!**

In [ ]:
plot.canvas.get_stats()

In [ ]:
plot["img"].cmap = "viridis"

In [ ]:
plot["img"].vmax=300